In [0]:
%sql
CREATE DATABASE IF NOT EXISTS disease_surveillance;


In [0]:
%sql
-- List all databases to confirm
SHOW DATABASES;


databaseName
default
disease_surveillance
diseasesurveillancedb
information_schema


In [0]:
%sql
-- Use the new database
USE disease_surveillance;

In [0]:
%sql
-- Create the Patients table
CREATE TABLE IF NOT EXISTS disease_surveillance.Patients (
    patient_id BIGINT GENERATED ALWAYS AS IDENTITY,  -- Auto-generated sequential ID
    patient_name STRING,                             -- Full name of the patient
    dob DATE,                                        -- Date of birth
    gender STRING,                                   -- Gender of the patient
    zipcode STRING,                                  -- Zipcode of the patient
    region_id BIGINT                                 -- Foreign key linking to Regions table
)
USING DELTA
PARTITIONED BY (zipcode);

-- Optimize the Patients table for query performance
OPTIMIZE disease_surveillance.Patients
ZORDER BY (patient_name);

-- Cache the Patients table
CACHE TABLE disease_surveillance.Patients;


In [0]:
%sql
-- Create the Diseases table
CREATE TABLE IF NOT EXISTS disease_surveillance.Diseases (
    disease_id BIGINT GENERATED ALWAYS AS IDENTITY,  -- Auto-generated unique ID
    disease_name STRING NOT NULL,                    -- Name of the disease
    description STRING                               -- Short description of the disease
)
USING DELTA;

-- Optimize the Diseases table for query performance
OPTIMIZE disease_surveillance.Diseases
ZORDER BY (disease_name);

-- Cache the Diseases table
CACHE TABLE disease_surveillance.Diseases;


In [0]:
%sql
-- Create the Cases table
CREATE TABLE IF NOT EXISTS disease_surveillance.Cases (
    case_id BIGINT GENERATED ALWAYS AS IDENTITY,    -- Auto-generated unique ID
    patient_id BIGINT NOT NULL,                     -- Foreign key linking to Patients table
    disease_id BIGINT NOT NULL,                     -- Foreign key linking to Diseases table
    diagnosis_date DATE NOT NULL,                   -- Date the disease was diagnosed
    severity STRING,                                -- Severity of the case
    notes STRING                                    -- Additional information (optional)
)
USING DELTA;

-- Optimize the Cases table for query performance
OPTIMIZE disease_surveillance.Cases
ZORDER BY (diagnosis_date);

-- Cache the Cases table
CACHE TABLE disease_surveillance.Cases;


In [0]:
%sql
-- Create the Vaccines table
CREATE TABLE IF NOT EXISTS disease_surveillance.Vaccines (
    vaccine_id BIGINT GENERATED ALWAYS AS IDENTITY,  -- Auto-generated unique ID
    vaccine_name STRING NOT NULL,                    -- Name of the vaccine
    manufacturer STRING,                             -- Name of the vaccine manufacturer
    description STRING                               -- Short description of the vaccine
)
USING DELTA;

-- Optimize the Vaccines table for query performance
OPTIMIZE disease_surveillance.Vaccines
ZORDER BY (vaccine_name);

-- Cache the Vaccines table
CACHE TABLE disease_surveillance.Vaccines;


In [0]:
%sql
-- Create the VaccineDiseases table (join table)
CREATE TABLE IF NOT EXISTS disease_surveillance.VaccineDiseases (
    vaccine_id BIGINT NOT NULL,                      -- Foreign key linking to Vaccines table
    disease_id BIGINT NOT NULL,                      -- Foreign key linking to Diseases table
    PRIMARY KEY (vaccine_id, disease_id)             -- Composite primary key
)
USING DELTA;

-- Optimize the VaccineDiseases table for query performance
OPTIMIZE disease_surveillance.VaccineDiseases
ZORDER BY (vaccine_id);

-- Cache the VaccineDiseases table
CACHE TABLE disease_surveillance.VaccineDiseases;


In [0]:
%sql
-- Create the Immunizations table
CREATE TABLE IF NOT EXISTS disease_surveillance.Immunizations (
    immunization_id BIGINT GENERATED ALWAYS AS IDENTITY,  -- Auto-generated unique ID
    patient_id BIGINT NOT NULL,                           -- Foreign key linking to Patients table
    vaccine_id BIGINT NOT NULL,                           -- Foreign key linking to Vaccines table
    immunization_date DATE NOT NULL,                      -- Date the vaccine was administered
    administered_by STRING                                -- Name of the healthcare provider (optional)
)
USING DELTA;

-- Optimize the Immunizations table for query performance
OPTIMIZE disease_surveillance.Immunizations
ZORDER BY (immunization_date);

-- Cache the Immunizations table
CACHE TABLE disease_surveillance.Immunizations;


In [0]:
%sql
-- Create the Regions table
CREATE TABLE IF NOT EXISTS disease_surveillance.Regions (
    region_id BIGINT GENERATED ALWAYS AS IDENTITY,  -- Auto-generated unique ID
    region_name STRING NOT NULL,                   -- Name of the region (e.g., "Manhattan")
    zipcode STRING                                 -- Zipcode associated with the region
)
USING DELTA
PARTITIONED BY (zipcode);

-- Optimize the Regions table for query performance
OPTIMIZE disease_surveillance.Regions
ZORDER BY (region_name);

-- Cache the Regions table
CACHE TABLE disease_surveillance.Regions;


In [0]:
%sql
show tables


database,tableName,isTemporary
disease_surveillance,cases,false
disease_surveillance,diseases,false
disease_surveillance,immunizations,false
disease_surveillance,patients,false
disease_surveillance,regions,false
disease_surveillance,vaccinediseases,false
disease_surveillance,vaccines,false
,_sqldf,true


In [0]:
%sql
-- Insert data into the Diseases table
INSERT INTO disease_surveillance.Diseases (disease_name, description)
VALUES
    ('Measles', 'A highly contagious viral disease'),
    ('Mumps', 'A viral disease causing swelling of salivary glands'),
    ('Rubella', 'A contagious viral disease also known as German measles'),
    ('Influenza', 'A viral infection causing fever, chills, and muscle pain'),
    ('COVID-19', 'A respiratory disease caused by the SARS-CoV-2 virus');


num_affected_rows,num_inserted_rows
5,5


In [0]:
%sql

select * from diseases

disease_id,disease_name,description
1,Measles,A highly contagious viral disease
2,Mumps,A viral disease causing swelling of salivary glands
3,Rubella,A contagious viral disease also known as German measles
4,Influenza,"A viral infection causing fever, chills, and muscle pain"
5,COVID-19,A respiratory disease caused by the SARS-CoV-2 virus


In [0]:
%sql
-- Insert data into the Vaccines table
INSERT INTO disease_surveillance.Vaccines (vaccine_name, manufacturer, description)
VALUES
    ('MMR', 'PharmaCo', 'A vaccine for Measles, Mumps, and Rubella'),
    ('Flu Shot', 'VaxCorp', 'Annual vaccine for influenza'),
    ('Pfizer-BioNTech', 'Pfizer', 'COVID-19 mRNA vaccine'),
    ('Moderna', 'Moderna', 'COVID-19 mRNA vaccine'),
    ('Johnson & Johnson', 'Janssen', 'COVID-19 viral vector vaccine');


num_affected_rows,num_inserted_rows
5,5


In [0]:
%sql
SELECT * FROM disease_surveillance.Vaccines;


vaccine_id,vaccine_name,manufacturer,description
1,MMR,PharmaCo,"A vaccine for Measles, Mumps, and Rubella"
2,Flu Shot,VaxCorp,Annual vaccine for influenza
3,Pfizer-BioNTech,Pfizer,COVID-19 mRNA vaccine
4,Moderna,Moderna,COVID-19 mRNA vaccine
5,Johnson & Johnson,Janssen,COVID-19 viral vector vaccine


In [0]:
%sql
-- Insert data into the VaccineDiseases table
INSERT INTO disease_surveillance.VaccineDiseases (vaccine_id, disease_id)
VALUES
    (1, 1),  -- MMR targets Measles
    (1, 2),  -- MMR targets Mumps
    (1, 3),  -- MMR targets Rubella
    (2, 4),  -- Flu Shot targets Influenza
    (3, 5),  -- Pfizer-BioNTech targets COVID-19
    (4, 5),  -- Moderna targets COVID-19
    (5, 5);  -- Johnson & Johnson targets COVID-19


num_affected_rows,num_inserted_rows
7,7


In [0]:
%sql
SELECT * FROM disease_surveillance.VaccineDiseases;


vaccine_id,disease_id
1,1
1,2
1,3
2,4
3,5
4,5
5,5


In [0]:
%sql
-- Insert data into the Regions table
INSERT INTO disease_surveillance.Regions (region_name, zipcode)
VALUES
    ('Manhattan', '10001'),
    ('Brooklyn', '11201'),
    ('Queens', '11373'),
    ('Bronx', '10451'),
    ('Staten Island', '10301');


num_affected_rows,num_inserted_rows
5,5


In [0]:
%sql 
select * from regions

region_id,region_name,zipcode
5,Staten Island,10301
1,Manhattan,10001
2,Brooklyn,11201
3,Queens,11373
4,Bronx,10451


1. List All Vaccines and the Diseases They Target


In [0]:
%sql
SELECT 
    v.vaccine_name AS Vaccine,
    d.disease_name AS Disease
FROM 
    disease_surveillance.VaccineDiseases vd
JOIN 
    disease_surveillance.Vaccines v ON vd.vaccine_id = v.vaccine_id
JOIN 
    disease_surveillance.Diseases d ON vd.disease_id = d.disease_id;


Vaccine,Disease
MMR,Rubella
Flu Shot,Influenza
Pfizer-BioNTech,COVID-19
Moderna,COVID-19
Johnson & Johnson,COVID-19
MMR,Mumps
MMR,Measles


2. List All Diseases and Their Corresponding Vaccines
(Similar to the query above, but focused on diseases first.)

In [0]:
%sql
SELECT 
    d.disease_name AS Disease,
    v.vaccine_name AS Vaccine
FROM 
    disease_surveillance.VaccineDiseases vd
JOIN 
    disease_surveillance.Diseases d ON vd.disease_id = d.disease_id
JOIN 
    disease_surveillance.Vaccines v ON vd.vaccine_id = v.vaccine_id;


Disease,Vaccine
Measles,MMR
Mumps,MMR
Rubella,MMR
Influenza,Flu Shot
COVID-19,Johnson & Johnson
COVID-19,Moderna
COVID-19,Pfizer-BioNTech


3. Find Diseases Without Any Vaccines


In [0]:
%sql
SELECT 
    d.disease_name AS Disease
FROM 
    disease_surveillance.Diseases d
LEFT JOIN 
    disease_surveillance.VaccineDiseases vd ON d.disease_id = vd.disease_id
WHERE 
    vd.vaccine_id IS NULL;


Disease


4. Find Vaccines Without Any Associated Diseases



In [0]:
%sql
SELECT 
    v.vaccine_name AS Vaccine
FROM 
    disease_surveillance.Vaccines v
LEFT JOIN 
    disease_surveillance.VaccineDiseases vd ON v.vaccine_id = vd.vaccine_id
WHERE 
    vd.disease_id IS NULL;


Vaccine


5. Count How Many Diseases Each Vaccine Targets


In [0]:
%sql
SELECT 
    v.vaccine_name AS Vaccine,
    COUNT(vd.disease_id) AS Disease_Count
FROM 
    disease_surveillance.VaccineDiseases vd
JOIN 
    disease_surveillance.Vaccines v ON vd.vaccine_id = v.vaccine_id
GROUP BY 
    v.vaccine_name;


Vaccine,Disease_Count
Johnson & Johnson,1
Moderna,1
Pfizer-BioNTech,1
Flu Shot,1
MMR,3


6. Count How Many Vaccines Exist for Each Disease


In [0]:
%sql
SELECT 
    d.disease_name AS Disease,
    COUNT(vd.vaccine_id) AS Vaccine_Count
FROM 
    disease_surveillance.VaccineDiseases vd
JOIN 
    disease_surveillance.Diseases d ON vd.disease_id = d.disease_id
GROUP BY 
    d.disease_name;


Disease,Vaccine_Count
Rubella,1
COVID-19,3
Measles,1
Mumps,1
Influenza,1


7. Get a Detailed Report of Vaccines, Their Manufacturers, and the Diseases They Target


In [0]:
%sql
SELECT 
    v.vaccine_name AS Vaccine,
    v.manufacturer AS Manufacturer,
    d.disease_name AS Disease
FROM 
    disease_surveillance.VaccineDiseases vd
JOIN 
    disease_surveillance.Vaccines v ON vd.vaccine_id = v.vaccine_id
JOIN 
    disease_surveillance.Diseases d ON vd.disease_id = d.disease_id;


Vaccine,Manufacturer,Disease
MMR,PharmaCo,Rubella
Flu Shot,VaxCorp,Influenza
Pfizer-BioNTech,Pfizer,COVID-19
Moderna,Moderna,COVID-19
Johnson & Johnson,Janssen,COVID-19
MMR,PharmaCo,Mumps
MMR,PharmaCo,Measles


8. List All Vaccines That Target COVID-19


In [0]:
%sql
SELECT 
    v.vaccine_name AS Vaccine
FROM 
    disease_surveillance.VaccineDiseases vd
JOIN 
    disease_surveillance.Vaccines v ON vd.vaccine_id = v.vaccine_id
JOIN 
    disease_surveillance.Diseases d ON vd.disease_id = d.disease_id
WHERE 
    d.disease_name = 'COVID-19';


Vaccine
Pfizer-BioNTech
Moderna
Johnson & Johnson


9. List All Diseases Targeted by the MMR Vaccine


In [0]:
%sql
SELECT 
    d.disease_name AS Disease
FROM 
    disease_surveillance.VaccineDiseases vd
JOIN 
    disease_surveillance.Vaccines v ON vd.vaccine_id = v.vaccine_id
JOIN 
    disease_surveillance.Diseases d ON vd.disease_id = d.disease_id
WHERE 
    v.vaccine_name = 'MMR';


Disease
Rubella
Mumps
Measles


10. Check for Any Duplicate Entries in VaccineDiseases


In [0]:
%sql
SELECT 
    vaccine_id, 
    disease_id, 
    COUNT(*) AS Entry_Count
FROM 
    disease_surveillance.VaccineDiseases
GROUP BY 
    vaccine_id, disease_id
HAVING 
    COUNT(*) > 1;


vaccine_id,disease_id,Entry_Count


11. Verify the Total Number of Vaccines and Diseases
(For quick verification of table population.)

In [0]:
%sql
-- Count total vaccines
SELECT COUNT(*) AS Total_Vaccines FROM disease_surveillance.Vaccines;

-- Count total diseases
SELECT COUNT(*) AS Total_Diseases FROM disease_surveillance.Diseases;

-- Count total vaccine-disease relationships
SELECT COUNT(*) AS Total_Vaccine_Disease_Relationships FROM disease_surveillance.VaccineDiseases;


Total_Vaccine_Disease_Relationships
7


12. Verify That Every Disease in VaccineDiseases Exists in the Diseases Table


In [0]:
%sql
SELECT 
    vd.disease_id
FROM 
    disease_surveillance.VaccineDiseases vd
LEFT JOIN 
    disease_surveillance.Diseases d ON vd.disease_id = d.disease_id
WHERE 
    d.disease_id IS NULL;


disease_id


13. Verify That Every Vaccine in VaccineDiseases Exists in the Vaccines Table


In [0]:
%sql
SELECT 
    vd.vaccine_id
FROM 
    disease_surveillance.VaccineDiseases vd
LEFT JOIN 
    disease_surveillance.Vaccines v ON vd.vaccine_id = v.vaccine_id
WHERE 
    v.vaccine_id IS NULL;


vaccine_id


In [0]:
%sql
DESCRIBE TABLE disease_surveillance.immunizations;

col_name,data_type,comment
immunization_id,bigint,null
patient_id,bigint,null
vaccine_id,bigint,null
immunization_date,date,null
administered_by,string,null


In [0]:
%sql
select * from patients


patient_id,patient_name,dob,gender,zipcode,region_id
17,Bobby Ponce,1985-12-07,Female,46958,null
28,Emily Cook,1961-04-26,Male,46958,null
61,Jamie Carrillo,1997-12-20,Male,46958,null
85,Jenna Sullivan,1944-07-21,Female,46958,null
90,Brian Newman,1972-01-21,Female,46958,null
125,Destiny Williams,2002-01-11,Male,46958,null
190,Jeffery Moss,2013-03-05,Female,46958,null
268,Maria Ramirez,2023-04-17,Female,46958,null
286,Zachary Shea,1954-07-15,Female,46958,null
313,Carolyn Nguyen,1975-09-16,Female,46958,null


In [0]:
%sql
select * from cases

case_id,patient_id,disease_id,diagnosis_date,severity,notes
1,7,2,2024-11-24,Severe,null
2,9,1,2024-11-01,Mild,null
3,10,5,2024-11-04,Mild,kjshdgh
4,11,4,2024-11-01,Severe,null
5,12,1,2011-11-11,Mild,kjsdhgkjhdgkjn
6,13,4,2024-11-03,Moderate,bkdjshgvpoeifpi hjdsgkjbg nbdnbg
7,532,18,2020-12-15,Severe,Place option success say prepare pick.
8,315,5,2021-12-25,Severe,Class eat determine open purpose.
9,390,18,2023-03-24,Severe,Want deep future drug.
10,932,8,2020-06-15,Severe,Fly nation change bring pressure half make.


In [0]:
%sql
select * from immunizations

immunization_id,patient_id,vaccine_id,immunization_date,administered_by
1,9,1,2024-11-10,John
2,11,4,2024-10-03,kjkjhkgggo
3,12,1,2018-11-24,mnbmnjshdg
4,13,3,2024-11-04,pppppppp
5,395,10,2022-11-18,Sara Downs
6,394,1,2021-01-24,Garrett Norton
7,21,4,2023-01-18,Kevin Fleming
8,210,3,2021-12-20,Jessica Miller
9,686,3,2022-09-23,Patricia Bautista
10,648,6,2021-05-21,Carmen Wright


In [0]:
%sql
SELECT * FROM disease_surveillance.Regions;


region_id,region_name,zipcode
29,Lake Cassandrashire,83256
19,Lake Nathanielview,71641
43,South Michaelmouth,78953
31,East Jameschester,84855
41,South Thomasshire,49750
14,North Kennethtown,21838
36,North Connieland,38589
10,East Shellyshire,09273
51,Lake Josephshire,03234
34,East Natashafort,33512
